<a href="https://colab.research.google.com/github/jpdelgad0/Ai-grader/blob/main/AES%20grader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install /content/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl >> none
!pip install /content/textstat-pypi/textstat-0.7.0-py3-none-any.whl >> none
!pip install "/content/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

Processing ./pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

from tqdm import tqdm
import json, string
from typing import List

import spacy
from spellchecker import SpellChecker

from collections import OrderedDict, Counter, defaultdict

import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import log_evaluation, early_stopping

import textstat
from textblob import TextBlob

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import mean_squared_error, cohen_kappa_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.simplefilter("ignore")

KeyboardInterrupt: 

In [ ]:
!unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

Archive:  /root/nltk_data/corpora/wordnet.zip
   creating: /root/nltk_data/corpora/wordnet/
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wordnet/noun.exc  
  inflating: /root/nltk_data/corpora/wordnet/verb.exc  
  inflating: /root/nltk_data/corpora/wordnet/README  
  inflating: /root/nltk_data/corpora/wordnet/index.sense  
  inflating: /root/nltk_data/corpora/wordnet/data.noun  
  inflating: /root/nltk_data/corpora/wordnet/data.adv  


In [ ]:
train_path='/content/learning-data/train.csv'
test_path='/content/learning-data/test.csv'
sub_path='/content/learning-data/sample_submission.csv'
n_splits=10
seed=42

In [ ]:
train=pd.read_csv(train_path)
train.head(5)

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [ ]:
test=pd.read_csv(test_path)
test.head(5)

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [ ]:
stopwords_list = [
    "a", "about", "above", "according", "across", "actually", "adj", "after", "afterwards", "again",
    "all", "almost", "along", "already", "also", "although", "always", "among", "amongst", "an",
    "am", "and", "another", "any", "anyhow", "anyone", "anything", "anywhere", "are", "aren",
    "aren't", "around", "as", "at", "be", "became", "because", "become", "becomes", "been",
    "beforehand", "begin", "being", "below", "beside", "besides", "between", "both", "but", "by",
    "can", "cannot", "can't", "caption", "co", "come", "could", "couldn", "couldn't", "did", "didn",
    "didn't", "do", "does", "doesn", "doesn't", "don", "don't", "down", "during", "each", "early",
    "eg", "either", "else", "elsewhere", "end", "ending", "enough", "etc", "even", "ever", "every",
    "everywhere", "except", "few", "for", "found", "from", "further", "had", "has", "hasn", "hasn't",
    "have", "haven", "haven't", "he", "hence", "her", "here", "hereafter", "hereby", "herein",
    "hereupon", "hers", "him", "his", "how", "however", "ie", "i.e.", "if", "in", "inc", "inc.",
    "indeed", "instead", "into", "is", "isn", "isn't", "it", "its", "itself", "last", "late",
    "later", "less", "let", "like", "likely", "ll", "ltd", "made", "make", "makes", "many", "may",
    "maybe", "me", "meantime", "meanwhile", "might", "miss", "more", "most", "mostly", "mr", "mrs",
    "much", "must", "my", "myself", "namely", "near", "neither", "never", "nevertheless", "new",
    "next", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "not", "now", "NULL",
    "of", "off", "often", "on", "once", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "per", "perhaps", "rather", "re", "said", "same",
    "say", "seem", "seemed", "seeming", "seems", "several", "she", "should", "shouldn", "shouldn't",
    "since", "so", "some", "still", "stop", "such", "taking", "ten", "than", "that", "the", "their",
    "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore",
    "therein", "thereupon", "these", "they", "this", "those", "though", "thousand", "through",
    "throughout", "thru", "thus", "to", "together", "too", "toward", "towards", "under", "unless",
    "unlike", "unlikely", "until", "up", "upon", "us", "use", "used", "using", "ve", "very", "via",
    "was", "wasn", "we", "well", "were", "weren", "weren't", "what", "whatever", "when", "whence",
    "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever",
    "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whomever", "whose",
    "why", "will", "with", "within", "without", "won", "would", "wouldn", "wouldn't", "yes", "yet",
    "you", "your", "yours", "yourself", "yourselves"
]

final_stopwords_list = list(set(stopwords.words('english')) | set(stopwords_list))
print(len(final_stopwords_list))

337


In [ ]:
contractions = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would",  ## --> he had or he would
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",   ## --> I had or I would
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",   ## --> It had or It would
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not", "oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",   ## --> It had or It would
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
}

contraction_pattern = re.compile('(%s)' % '|'.join(contractions .keys()))

def expand_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contraction_pattern.sub(replace, text)

In [ ]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    text = re.sub(r'\s+', ' ', text)
    return text

def remove_stopwords (text):
    tokens = nltk.word_tokenize(text)
    stop_words = set(final_stopwords_list)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def lemmatization(text):
    words = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if len(word) > 1]
    return ' '.join(lemmatized_words)

def preprocess_data(text):
    text = text.lower() # Convert words to lowercase
    text = re.sub(r'<.*?>', '', text) # Remove HTML
    text = re.sub("@\w+", '',text)     # Delete strings starting with @
    text = re.sub("\d+", '',text)
    text = re.sub("'\d+", '',text) # Delete Numbers
    text = re.sub("http\w+", '',text) # Delete URL
    text = text.replace(u'\xa0',' ') # Remove \xa0
    text = re.sub(r'_+', ' ', text)
    text = re.sub(r"\s+", " ", text)
    text = expand_contractions(text)
    # Replace consecutive commas and periods with one comma and period character
    text = re.sub(r"\.+", ".", text)
    text = re.sub(r"\,+", ",", text)
    text = re.sub(r"\s+", " ", text) # Replace consecutive empty spaces with a single space character
    text = text.strip()
    return text

def preprocess_with_contractions_and_punctuation_removal(text):
    text = text.lower() # Convert words to lowercase
    text = re.sub(r'<.*?>', '', text)  # Remove HTML
    text = expand_contractions(text)
    text = remove_stopwords(text) # Remove stopwords
    text = re.sub("@\w+", '',text) # Delete strings starting with @
    text = text.replace(u'\xa0',' ') # Remove \xa0
    text = re.sub("'\d+", '',text) # Delete Numbers
    text = re.sub("\d+", '',text)
    text = re.sub(r'_+', ' ', text)
    text = re.sub("http\w+", '',text)     # Delete URL
    text = remove_punctuation(text) # Remove punctuation
    text = re.sub(r"\s+", " ", text) # Replace consecutive empty spaces with a single space character
    text = lemmatization(text) # Lemmatizing
    text = text.strip()
    return text

In [ ]:
def remove_duplicates(text):
    sentences = text.split('. ')

    # Use an OrderedDict to remove duplicates while preserving order
    unique_sentences = list(OrderedDict.fromkeys(sentences))

    # Join the unique sentences back into a single string
    result = '. '.join(unique_sentences)

    # Ensure the final sentence ends with a period if it originally did
    if text.endswith('.'):
        result += '.'

    return result

def extract_sentences(text):
    # Use a regular expression to split the text into sentences
    # This will handle periods, exclamation marks, and question marks as sentence terminators
    sentences = re.split(r'[.!?]+\s*', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

def extract_words(text):
    words = re.findall(r"\w+(?:[-']\w+)*", text)
    return words

def preprocessing_for_paragraphs(text):
    # If before /n/n is not a mark, this is not the end of a paragraph
    text = re.sub(r'(?<![\.\!\?])\n\n', ' ', text)

    #If after \n\n is an normal case, replace with space
    text = re.sub(r'\n\n([a-z])', ' ', text)

    return text.strip()

def extract_paragraphs(text):
    processed_text = preprocessing_for_paragraphs(text)
    paragraphs = processed_text.split('\n\n')

    return paragraphs

In [ ]:
columns = [(pl.col("full_text").apply(extract_paragraphs).alias("paragraph"))]
train = pl.from_pandas(train).with_columns([pl.col("full_text").apply(remove_duplicates)])
test = pl.from_pandas(test).with_columns([pl.col("full_text").apply(remove_duplicates)])

train = train.with_columns(columns)
test = test.with_columns(columns)

In [ ]:
# paragraph features
def paragraph_preprocess(temp_df):
    # Expand the paragraph list into several lines of data
    temp_df = temp_df.explode('paragraph')

    # Paragraph preprocessing
    temp_df = temp_df.with_columns(pl.col('paragraph').map_elements(preprocess_data))

    # Calculate the length of each paragraph
    temp_df = temp_df.with_columns(
        pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"),
        pl.col('paragraph').map_elements(lambda x: len(extract_sentences(x))).alias('paragraph_sentence_cnt'),
        pl.col('paragraph').map_elements(lambda x: len(extract_words(x))).alias('paragraph_word_cnt'),
        pl.col('paragraph').map_elements(lambda x: len(set(extract_words(x)))).alias('paragraph_unique_word_cnt')
    )
    return temp_df

In [ ]:
length_ranges = [(1, 100), (101, 200), (201, 300), (301, 400), (401, 500), (501, 600), (601, 800)]
paragraph_fea = ['paragraph_len', 'paragraph_sentence_cnt', 'paragraph_word_cnt', 'paragraph_unique_word_cnt']

def paragraph_feature_engineering(train_df):
    count_aggs = [
        pl.col('paragraph').filter((pl.col('paragraph_len') >= start) & (pl.col('paragraph_len') <= end)).count().alias(f"paragraph_len_between_{start}_{end}_cnt")
        for start, end in length_ranges
    ]

    aggs = [
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_len_geq_{i}_cnt") for i in [100,150,200,300,350,400,500,600,700] ],
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea],
        ]

    df = train_df.group_by(['essay_id'], maintain_order=True).agg(count_aggs + aggs).sort("essay_id")
    df = df.to_pandas()
    return df

In [ ]:
train_preprocessed = paragraph_preprocess(train)
train_features = paragraph_feature_engineering(train_preprocessed)

# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))

print(f'Features Number: {len(feature_names)}')
train_features.head(5)

Features Number: 32


,essay_id,paragraph_len_between_1_100_cnt,paragraph_len_between_101_200_cnt,paragraph_len_between_201_300_cnt,paragraph_len_between_301_400_cnt,paragraph_len_between_401_500_cnt,paragraph_len_between_501_600_cnt,paragraph_len_between_601_800_cnt,paragraph_len_geq_100_cnt,paragraph_len_geq_150_cnt,...,paragraph_word_cnt_mean,paragraph_unique_word_cnt_mean,paragraph_len_sum,paragraph_sentence_cnt_sum,paragraph_word_cnt_sum,paragraph_unique_word_cnt_sum,paragraph_len_min,paragraph_sentence_cnt_min,paragraph_word_cnt_min,paragraph_unique_word_cnt_min
0,000d118,0,0,0,0,0,0,0,1,1,...,501.00,220.0,2645,13,501,220,2645,13,501,220
1,000fe60,0,1,1,2,1,0,0,5,5,...,67.00,45.6,1662,21,335,228,184,2,37,29
2,001ab80,0,0,0,0,1,1,0,4,4,...,137.50,84.5,3062,24,550,338,476,4,85,60
3,001bdc0,0,0,0,2,0,0,0,4,4,...,111.75,76.5,2674,21,447,306,367,2,63,51
4,002ba53,1,0,0,1,0,2,1,4,4,...,72.80,46.6,2120,15,364,233,17,1,3,3


In [ ]:
def sentence_preprocess(temp_df):
    # Preprocess full_text and use periods to segment sentences in the text
    temp_df = temp_df.with_columns( pl.col('full_text').map_elements(preprocess_data).map_elements(extract_sentences).alias("sentences"))
    temp_df = temp_df.explode('sentences')

    temp_df = temp_df.with_columns(
        pl.col('sentences').map_elements(lambda x: len(x)).alias("sentence_len"),
        pl.col('sentences').map_elements(lambda x: len(extract_words(x))).alias("sentence_word_cnt"),
        pl.col('sentences').map_elements(lambda x: len(set(extract_words(x)))).alias("sentence_unique_word_cnt")
    )

    return temp_df

In [ ]:
sentence_length_ranges = [(1, 50), (51, 100), (101, 150), (151, 300)]
sentence_fea = ['sentence_len','sentence_word_cnt', 'sentence_unique_word_cnt']

def sentence_feature_engineering(train_tmp):

    count_aggs = [
        pl.col('sentences').filter((pl.col('sentence_len') >= start) & (pl.col('sentence_len') <= end)).count().alias(f"sentence_len_between_{start}_{end}_cnt")
        for start, end in sentence_length_ranges
    ]

    aggs = [
        *[pl.col('sentences').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_len_geq_{i}_cnt") for i in [50,100,150,300] ],
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
    ]

    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(count_aggs + aggs).sort("essay_id")
    df = df.to_pandas()

    return df

In [ ]:
train_preprocessed = sentence_preprocess(train)
train_features = train_features.merge(sentence_feature_engineering(train_preprocessed), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))
print('Features Number: ',len(feature_names))
train_features.head(5)

Features Number:  46


,essay_id,paragraph_len_between_1_100_cnt,paragraph_len_between_101_200_cnt,paragraph_len_between_201_300_cnt,paragraph_len_between_301_400_cnt,paragraph_len_between_401_500_cnt,paragraph_len_between_501_600_cnt,paragraph_len_between_601_800_cnt,paragraph_len_geq_100_cnt,paragraph_len_geq_150_cnt,...,sentence_len_geq_50_cnt,sentence_len_geq_100_cnt,sentence_len_geq_150_cnt,sentence_len_geq_300_cnt,sentence_len_max,sentence_word_cnt_max,sentence_unique_word_cnt_max,sentence_len_mean,sentence_word_cnt_mean,sentence_unique_word_cnt_mean
0,000d118,0,0,0,0,0,0,0,1,1,...,11,11,7,3,593,128,70,201.538462,38.538462,28.692308
1,000fe60,0,1,1,2,1,0,0,5,5,...,16,7,1,0,249,48,37,77.380952,15.952381,14.190476
2,001ab80,0,0,0,0,1,1,0,4,4,...,24,15,7,0,236,46,39,125.750000,22.916667,20.666667
3,001bdc0,0,0,0,2,0,0,0,4,4,...,20,11,7,0,209,35,32,125.619048,21.285714,19.285714
4,002ba53,1,0,0,1,0,2,1,4,4,...,14,10,5,1,440,76,50,139.866667,24.266667,19.733333


In [ ]:
def word_preprocess(temp_df):
    # Preprocess full_text and use spaces to separate words from the text
    temp_df = temp_df.with_columns(pl.col('full_text').map_elements(preprocess_data).map_elements(extract_words).alias('word'))
    temp_df = temp_df.explode('word')

    # Calculate the length of each word
    temp_df = temp_df.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))

    # Delete data with a word length of 0
    temp_df = temp_df.filter(pl.col('word_len') != 0)

    return temp_df

In [ ]:
word_length_ranges = [(1, 5), (6, 10), (11, 15)]

def word_feature_engineering(train_tmp):

    count_aggs = [
        pl.col('word').filter((pl.col('word_len') >= start) & (pl.col('word_len') <= end)).count().alias(f"word_len_between_{start}_{end}_cnt")
        for start, end in word_length_ranges
    ]
    aggs = [
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_len_geq_{i+1}_cnt") for i in range(15) ],

        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        ]

    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(count_aggs + aggs).sort("essay_id")

    df = df.to_pandas()

    return df

In [ ]:
train_preprocessed = word_preprocess(train)

# Merge the newly generated feature data with the previously generated feature data
train_features = train_features.merge(word_feature_engineering(train_preprocessed), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))
print('Features Number: ',len(feature_names))
train_features.head(5)

In [ ]:
nltk.download('punkt')
character_tfidf_vectorizer = TfidfVectorizer(
            tokenizer = lambda x: x,
            preprocessor = lambda x: x,
            token_pattern = None,
            strip_accents = 'unicode',
            analyzer = 'word',
            ngram_range = (1,3),
            min_df = 0.1,
            max_df = 0.95,
            sublinear_tf = True,
)
# Processed text
processed_text = train.to_pandas()["full_text"].progress_apply(lambda x: preprocess_with_contractions_and_punctuation_removal(x))
train_tfidf = character_tfidf_vectorizer.fit_transform([i for i in processed_text])
character_tfidf_feature_names = character_tfidf_vectorizer.get_feature_names_out()
tfidf_features = pd.DataFrame(train_tfidf.toarray(), columns=[f"tfidf_{name}" for name in character_tfidf_feature_names ])
tfidf_features['essay_id'] = train_features['essay_id']
train_features = train_features.merge(tfidf_features, on='essay_id', how='left')

print('Character tf-idf features:')
print(character_tfidf_feature_names[0:100])

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))
print('Features Number: ',len(feature_names))
train_features.head(5)

In [ ]:
word_tfidf_vectorizer = TfidfVectorizer(
    preprocessor = lambda x: x,
    strip_accents = 'unicode',
    analyzer = 'word',
    ngram_range = (1, 3),
    min_df = 0.05,
    max_df = 0.85,
    sublinear_tf = True,
    stop_words = final_stopwords_list,
)

# Fit all datasets into TfidfVectorizer
train_tfidf = word_tfidf_vectorizer.fit_transform([i for i in processed_text])
word_tfidf_feature_names = word_tfidf_vectorizer.get_feature_names_out()
print('Word tf-idf features:')
print(word_tfidf_feature_names[0:100])

df = pd.DataFrame(train_tfidf.toarray(), columns=[f"tfidf_{name}" for name in word_tfidf_feature_names])
df['essay_id'] = train_features['essay_id']
train_features = train_features.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))
print('Features Number: ',len(feature_names))

train_features.head(5)

In [ ]:
tfidf_w_columns = [ f'tfidf_{i}' for i in word_tfidf_feature_names]
test_tfidf = train_features[tfidf_w_columns]
test_tfidf[test_tfidf > 0].agg(['count', 'min', 'max', 'std', 'mean']).T.to_csv('tfidf_test.csv')

In [ ]:
# Create test copy of dataframe
kmean_test = train_features[tfidf_w_columns]

In [ ]:
kmeans = KMeans(n_clusters=7, random_state=42)

# Fit the model to the data
kmeans.fit(kmean_test)

# Predict the clusters for the data points
labels = kmeans.labels_

# Get the centroids
centroids = kmeans.cluster_centers_

# Calculate the distance to the centroid
distances = np.sqrt(((kmean_test - centroids[labels]) ** 2).sum(axis=1))

cosine_distances_to_centroid = [
    cosine_distances([kmean_test.iloc[i]], [centroids[label]])[0][0]
    for i, label in enumerate(labels)
]

# Add the distances to the DataFrame
kmean_test['DistanceToCentroid'] = distances
kmean_test['CosineDistanceToCentroid'] = cosine_distances_to_centroid

train_features['DistanceToCentroid'] = kmean_test['DistanceToCentroid']
train_features['CosineDistanceToCentroid'] = kmean_test['CosineDistanceToCentroid']

In [ ]:
count_vectorizer = CountVectorizer(
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.05,
            max_df=0.85,
)

train_count = count_vectorizer.fit_transform([i for i in processed_text])

dense_matrix = train_count.toarray()
word_count_feature_names = count_vectorizer.get_feature_names_out()
print(word_count_feature_names[0:100])

df = pd.DataFrame(dense_matrix,  columns=[f"count_{name}" for name in word_count_feature_names])
df['essay_id'] = train_features['essay_id']
train_features = train_features.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))
print('Features Number: ',len(feature_names))
train_features.head()

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
class Preprocessor:
    def __init__(self) -> None:
        self.STOP_WORDS = set(final_stopwords_list)
        self.spellchecker = SpellChecker()

    def spelling(self, text):
        text_2 = re.sub(r'[^\w\s]', ' ', text)
        amount_miss = len(list(self.spellchecker.unknown(text_2.split())))
        return amount_miss

    def find_wrong_punctuation(self, text):
        punctuations = ['.', ',', ';', '?', '!', ':']
        lowercase_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
        uppercase_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

        # find punctuation in the text
        wrong_punctuations = 0
        length = len(text)

        for i in range(length):
            if text[i] in punctuations and i < length - 1:
                if text[i + 1] in lowercase_list or text[i + 1] in uppercase_list:
                    wrong_punctuations += 1

        return wrong_punctuations

    def noun_verb_adj_adv_adp_others(self, text):
        doc = nlp(text)
        pos_counts = doc.count_by(spacy.attrs.POS)
        nouns = 0
        verbs = 0
        adj = 0
        adv = 0
        adp_conj = 0
        others = 0

        for pos_id, count in pos_counts.items():
            pos_tag = doc.vocab.strings[pos_id]
            if pos_tag in ['NOUN', 'PROPN', 'PRON']:
                nouns += count
            elif pos_tag in ['VERB', 'AUX']:
                verbs += count
            elif pos_tag == 'ADJ':
                adj += count
            elif pos_tag == 'ADV':
                adv += count
            elif pos_tag in ['ADP', 'CONJ']:
                adp_conj += count
            else:
                others += count

        return nouns, verbs, adj, adv, adp_conj, others

    def count_sym(self, text, sym):
        sym_count = 0
        for l in text:
            if l == sym:
                sym_count += 1
        return sym_count

    def lexical_diversity(self,text):
        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Calculate the number of unique words (types) and total number of words (tokens)
        num_types = len(set(words))
        num_tokens = len(words)

        # Calculate the Type-Token Ratio (TTR)
        ttr = num_types / num_tokens

        return ttr

    def calculate_collocation_diversity(self,text):
        tokens = nltk.word_tokenize(text)
        finder = BigramCollocationFinder.from_words(tokens)
        return len(finder.score_ngrams(BigramAssocMeasures.mi_like)) / float(len(tokens))

    def calculate_collocation_strength(self,text):
        tokens = nltk.word_tokenize(text)
        finder = BigramCollocationFinder.from_words(tokens)
        collocations = finder.nbest(BigramAssocMeasures.mi_like, 10)  # Get top 10 collocations
        return sum(score for bigram, score in finder.score_ngrams(BigramAssocMeasures.mi_like)) / float(len(collocations))

    def run(self, data: pd.DataFrame, mode:str) -> pd.DataFrame:

          # preprocessing the text
        data["processed_text"] = data["full_text"].apply(lambda x: preprocess_with_contractions_and_punctuation_removal(x))


        data[['nouns', 'verbs', 'adj', 'adv', 'adp_conj', 'others']] = data['processed_text'].apply(lambda x: pd.Series(self.noun_verb_adj_adv_adp_others(x)))

         # distinct word count
        data['distinct_word_count'] = data['processed_text'].apply(lambda x: len(set(word_tokenize(x))))

        # coleman
        data['coleman_liau'] = data['processed_text'].apply(lambda x: textstat.coleman_liau_index(x))

        # Text tokenization
        data["text_tokens"] = data["processed_text"].apply(lambda x: word_tokenize(x))

         # lexical diversity
        data['lexical_diversity'] = data['full_text'].apply(lambda x: self.lexical_diversity(x))

        # collocation diversity
        data['collocation_diversity'] = data['processed_text'].apply(lambda x: self.calculate_collocation_diversity(x))

        # collocation strength
        data['collocation_strength'] = data['processed_text'].apply(lambda x: self.calculate_collocation_strength(x))

        # essay length
        data["text_length"] = data["processed_text"].apply(lambda x: len(x))
        data["full_text_length"] = data["full_text"].apply(lambda x: len(x))

        # essay word count
        data["word_count"] = data["text_tokens"].apply(lambda x: len(x))

        # essay unique word count
        data["unique_word_count"] = data["text_tokens"].apply(lambda x: len(set(x)))

        # essay sentence count
        data["sentence_count"] = data["full_text"].apply(lambda x: len(extract_sentences(x)))

        # essay paragraph count
        data["paragraph_count"] = data["full_text"].apply(lambda x: len(extract_paragraphs(x)))

        # count misspelling
        data["splling_err_num"] = data["processed_text"].progress_apply(self.spelling)
        data["splling_err_ratio"] = data["splling_err_num"] / data["text_length"]

        # ratio fullstop / text_length
        data["fullstop_ratio"] = data["full_text"].apply(lambda x: x.count(".")/len(x))

        # ratio comma / text_length
        data["comma_ratio"] = data["full_text"].apply(lambda x: x.count(",")/len(x))

        return data

In [ ]:
preprocessor = Preprocessor()
tmp = preprocessor.run(train.to_pandas(), mode="train")
train_features = train_features.merge(tmp, on='essay_id', how='left')
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_features.columns))

print('Features Number: ',len(feature_names))
train_features.head(5)

In [ ]:
tmp = paragraph_preprocess(test)
test_features = paragraph_feature_engineering(tmp)

# Sentence
tmp = sentence_preprocess(test)
test_features = test_features.merge(sentence_feature_engineering(tmp), on='essay_id', how='left')

# Word
tmp = word_preprocess(test)
test_features = test_features.merge(word_feature_engineering(tmp), on='essay_id', how='left')

# Character Tfidf
processed_text = test.to_pandas()["full_text"].progress_apply(lambda x: preprocess_with_contractions_and_punctuation_removal(x))

test_tfidf = character_tfidf_vectorizer.transform([i for i in processed_text])
feature_names = character_tfidf_vectorizer.get_feature_names_out()
df = pd.DataFrame(test_tfidf.toarray(), columns=[f'tfidf_{name}' for name in feature_names])
df['essay_id'] = test_features['essay_id']
test_features = test_features.merge(df, on='essay_id', how='left')

# Word Tfidf
test_tfidf = word_tfidf_vectorizer.transform([i for i in processed_text])
feature_names = word_tfidf_vectorizer.get_feature_names_out()
df = pd.DataFrame(test_tfidf.toarray(), columns=[f'tfidf_{name}' for name in feature_names])
df['essay_id'] = test_features['essay_id']
test_features = test_features.merge(df, on='essay_id', how='left')


# Fit the model to the data
kmean_test = test_features[tfidf_w_columns]
labels = kmeans.predict(kmean_test)
centroids = kmeans.cluster_centers_
print(len(centroids))
distances = np.sqrt(((kmean_test - centroids[labels]) ** 2).sum(axis=1))
cosine_distances_to_centroid = [
    cosine_distances([kmean_test.iloc[i]], [centroids[label]])[0][0]
    for i, label in enumerate(labels)
]
# Add the distances to the DataFrame
kmean_test['DistanceToCentroid'] = distances
kmean_test['CosineDistanceToCentroid'] = cosine_distances_to_centroid

test_features['DistanceToCentroid'] = kmean_test['DistanceToCentroid']
test_features['CosineDistanceToCentroid'] = kmean_test['CosineDistanceToCentroid']


# Word vectorize
test_count = count_vectorizer.transform([i for i in processed_text])
feature_names = count_vectorizer.get_feature_names_out()
test_count_df = pd.DataFrame(test_count.toarray(), columns=[f'count_{name}' for name in feature_names])
test_count_df['essay_id'] = test_features['essay_id']
test_features = test_features.merge(test_count_df, on='essay_id', how='left')


# Extra feature
tmp = preprocessor.run(test.to_pandas(), mode="train")
test_features = test_features.merge(tmp, on='essay_id', how='left')

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_features.columns))
print('Features number: ',len(feature_names))
test_features.head(3)

In [ ]:
train_features['score'] = train['score']

In [ ]:
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

for i, (_, val_index) in enumerate(skf.split(train_features, train_features["score"])):
    train_features.loc[val_index, "fold"] = i


print(train_features.shape)

In [ ]:
target = "score"
train_drop_columns = ["essay_id", "fold", "full_text", "paragraph", "text_tokens", "processed_text"] + [target]
test_drop_columns = ["essay_id", "full_text", "paragraph", "text_tokens", "processed_text"]

In [ ]:
def sanitize_feature_names(df):
    sanitized_columns = {col: re.sub(r'[^\w]', '_', col) for col in df.columns}
    df.rename(columns=sanitized_columns, inplace=True)
    return df

In [ ]:
train_features = sanitize_feature_names(train_features)
test_features = sanitize_feature_names(test_features)

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = (y_true + a).round()
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")

    return 'QWK', qwk, True

def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2 * np.sum((preds-labels)**2)
    g = 1/2 * np.sum((preds-a)**2 + b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg / g**2) * len(labels)
    hess = np.ones(len(labels))

    return grad, hess

def qwk_param_calc(y):
    a = y.mean()
    b = (y ** 2).mean() - a ** 2

    return np.round(a, 4), np.round(b, 4)

In [ ]:
models = []

callbacks = [
    lgb.log_evaluation(period=25),
    lgb.early_stopping(stopping_rounds=75, first_metric_only=True)
]

for fold in range(n_splits):

    model = lgb.LGBMRegressor(
                            objective = qwk_obj,
                            metrics = 'None',
                            learning_rate = 0.02,
                            max_depth = 10,
                            num_leaves = 15,
                            colsample_bytree = 0.5,
                            reg_alpha = 0.5,
                            reg_lambda = 1,
                            n_estimators = 1024,
                            random_state = seed,
                            extra_trees=True,
                            class_weight='balanced',
                            verbosity = - 1
                            )

    a, b = qwk_param_calc(train_features[train_features["fold"] != fold]["score"])

    # Take out the training and validation sets for 5 kfold segmentation separately
    X_train = train_features[train_features["fold"] != fold].drop(columns=train_drop_columns)
    y_train = train_features[train_features["fold"] != fold]["score"] - a

    X_eval = train_features[train_features["fold"] == fold].drop(columns=train_drop_columns)
    y_eval = train_features[train_features["fold"] == fold]["score"] - a

    print('\nFold_{} Training ================================\n'.format(fold+1))
    print(f"Fold {fold} a: {a}  ;;  b: {b}")

    # Training model
    lgb_model = model.fit(
                            X_train, y_train,
                            eval_names = ['train', 'valid'],
                            eval_set = [(X_train, y_train), (X_eval, y_eval)],
                            eval_metric = quadratic_weighted_kappa,
                            callbacks = callbacks
                        )

    models.append(model)

In [ ]:
preds, trues = [], []

for fold, model in enumerate(models):
    X_eval_cv = train_features[train_features["fold"] == fold].drop(columns=train_drop_columns)
    y_eval_cv = train_features[train_features["fold"] == fold]["score"]

    pred = model.predict(X_eval_cv) + a

    pred[pred < 1] = 1
    pred[pred > 6] = 6

    trues.extend(y_eval_cv)
    preds.extend(np.round(pred, 0))

    v_score = cohen_kappa_score(trues, preds, weights="quadratic")

    print(f"Validation score {fold} : {v_score}")

v_score = cohen_kappa_score(trues, preds, weights="quadratic")
print(f"Validation score : {v_score}")

In [ ]:
def analyze_preds(trues, preds):
    # Create dataframe
    model_prec = pd.DataFrame([trues, preds]).T
    model_prec.rename(columns = {0: 'trues', 1: 'preds'}, inplace=True)
    model_prec['correct'] = model_prec['trues'] == model_prec['preds']
    model_prec['count'] = model_prec.groupby('trues')['trues'].transform('count')
    model_prec['correct_count'] = model_prec.groupby('trues')['correct'].transform('sum')
    model_prec['correct_rate'] = model_prec['correct_count'] / model_prec['count']

    # Print binary correction rate
    print(model_prec[['trues', 'correct_rate', 'correct_count', 'count']].drop_duplicates().sort_values(by='trues'))

    # Plot predictions by score
    def plot_model(ax, counts, true):
        bars = ax.bar(counts.index, counts.values, color='skyblue')

        # Find the index of the column with the specified label
        highlight_index = counts.index.get_loc(true)

        # Highlight the specified column
        bars[highlight_index].set_color('orange')

        ax.set_xlabel('Predicted Values')
        ax.set_ylabel('Count')
        ax.set_title("Score " + str(true))

    score_list = [1,2,3,4,5,6]
    test_pred_by_score = [model_prec[model_prec['trues'] ==  score]['preds'].value_counts() for score in score_list]

    # Create a figure and six subplots arranged in a 2x3 grid
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plot_model(axs[0, 0], test_pred_by_score[0], 1)
    plot_model(axs[0, 1], test_pred_by_score[1], 2)
    plot_model(axs[0, 2], test_pred_by_score[2], 3)
    plot_model(axs[1, 0], test_pred_by_score[3], 4)
    plot_model(axs[1, 1], test_pred_by_score[4], 5)
    plot_model(axs[1, 2], test_pred_by_score[5], 6)

analyze_preds(trues, preds)

In [ ]:
top_features = ['splling_err_ratio', 'lexical_diversity', 'paragraph_count', 'word_count', 'DistanceToCentroid', 'word_len_sum']

In [ ]:
ax = lgb.plot_importance(models[1], figsize=(20, 200), importance_type="split")
plt.show()

In [ ]:
def show_feat_use(feat):
    ax = lgb.plot_split_value_histogram(models[2], figsize=(20, 8), feature=feat)
    plt.show()

In [ ]:
show_feat_use('tfidf_luke')

In [ ]:
def check_model_fit(model_id):
    ax = lgb.plot_metric(models[model_id])
    plt.show()

In [ ]:
check_model_fit(2)

In [ ]:
for i in range(len(models)):
    models[i].booster_.save_model(f'model_{i}.txt')

In [ ]:
test_features

In [ ]:
preds = []
for fold, model in enumerate(models):
    X_eval_cv = test_features.drop(columns=test_drop_columns)
    pred = model.predict(X_eval_cv) + a
    pred[pred < 1] = 1
    pred[pred > 6] = 6
    preds.append(pred)

# Combining the 5 model results
for i, pred in enumerate(preds):
    test_features[f"score_pred_{i}"] = pred
test_features["score"] = np.round(test_features[[f"score_pred_{fold}" for fold in range(n_splits)]].mean(axis=1),0).astype('int32')

# Submit to CSV
test_features[["essay_id", "score"]].to_csv("submission.csv", index=False)